# trialing a pipeline 

Here we want to create a trial pipleine , in this trial pipeline we will have random integers the goal is to get out model to output some scores for a couple pipes basically for now they can be random integers 

In [2]:
# example model to run for now just to get an idea 
import torch
import torch.nn as nn


class TowerOne(nn.Module):
    def __init__(self):
        super(TowerOne, self).__init__()
        
        # First layer: input 64 features, output 32 features
        self.fc1 = nn.Linear(64, 32)
        
        # Second layer: input 32 features, output 16 features
        self.fc2 = nn.Linear(32, 16)
        
        # Third layer: input 16 features, output 3 features (final output)
        self.fc3 = nn.Linear(16, 3)
        
        # Activation function (ReLU) applied after first and second layers
        self.relu = nn.ReLU()

    def forward(self, x):
        # Pass input through the first layer, followed by ReLU
        x = self.relu(self.fc1(x))
        
        # Pass through the second layer, followed by ReLU
        x = self.relu(self.fc2(x))
        
        # Pass through the final layer to get 3 output features
        x = self.fc3(x)
        
        return x

class TowerTwo(nn.Module):
    def __init__(self):
        super(TowerTwo, self).__init__()
        self.fc = nn.Linear(64, 3)
    def forward(self, x):
        x = self.fc(x)
        return x


In [14]:
# Generate random inputs
video = torch.rand(1, 64)
video = torch.rand(1, 64)
video = torch.rand(1, 64)
video = torch.rand(1, 64)


query = torch.rand(1, 3)   # Input for TowerTwo

# Initialize models
tower_one = TowerOne()
tower_two = TowerTwo()

# Forward pass through both towers
output_one = tower_one(video)  # Pass video through TowerOne
output_two = tower_two(query)  # Pass query through TowerTwo

# Compute cosine similarity between the outputs
score = nn.functional.cosine_similarity(output_one, output_two, dim=1)

# Define target and compute loss (mean squared error)
target = torch.tensor([1.0])  # The target cosine similarity
loss = (score - target).pow(2).mean()

# Backpropagation
loss.backward()

# Print outputs for debugging
print("Video input:", video)
print("Query input:", query)
print("Output from TowerOne:", output_one)
print("Output from TowerTwo:", output_two)
print("Cosine similarity score:", score)
print("Loss:", loss)


Video input: tensor([[0.4139, 0.7380, 0.2999, 0.3325, 0.0293, 0.4814, 0.3391, 0.9069, 0.7763,
         0.0498, 0.5203, 0.0422, 0.3766, 0.6510, 0.6568, 0.4529, 0.9977, 0.2522,
         0.9066, 0.0753, 0.8177, 0.1161, 0.6997, 0.4761, 0.5777, 0.1587, 0.9614,
         0.5546, 0.4156, 0.5668, 0.9692, 0.1272, 0.5269, 0.8872, 0.6158, 0.8411,
         0.7299, 0.1364, 0.8546, 0.0254, 0.3160, 0.8907, 0.3916, 0.0663, 0.2259,
         0.9908, 0.0894, 0.6401, 0.2493, 0.8152, 0.6204, 0.2751, 0.6881, 0.4009,
         0.7173, 0.4172, 0.1987, 0.7279, 0.9791, 0.4464, 0.2679, 0.2666, 0.5608,
         0.2206]])
Query input: tensor([[0.2647, 0.4808, 0.9866]])
Output from TowerOne: tensor([[-0.0077,  0.0032, -0.2654]], grad_fn=<AddmmBackward0>)
Output from TowerTwo: tensor([[-0.4474,  0.2700,  0.4150]], grad_fn=<AddmmBackward0>)
Cosine similarity score: tensor([-0.5973], grad_fn=<SumBackward1>)
Loss: tensor(2.5513, grad_fn=<MeanBackward0>)


In [18]:
import torch
import torch.nn as nn

# Define margin for triplet loss
margin = 1.0


positive_video1 = torch.rand(1, 64)
positive_video2 = torch.rand(1, 64)
negative_video1 = torch.rand(1, 64)
negative_video2 = torch.rand(1, 64)

anchor_query = torch.rand(1, 64)  # Input for TowerTwo (optional for your task)

# Initialize models
tower_one = TowerOne()
tower_two = TowerTwo()

# Forward pass through the tower for each video
anchor_output = tower_one(anchor_query)          # Anchor
positive_output1 = tower_two(positive_video1)    # Positive sample 1
positive_output2 = tower_two(positive_video2)    # Positive sample 2
negative_output1 = tower_two(negative_video1)    # Negative sample 1
negative_output2 = tower_two(negative_video2)    # Negative sample 2

# Cosine similarity between anchor and positives
cosine_sim_positive1 = nn.functional.cosine_similarity(anchor_output, positive_output1, dim=1)
cosine_sim_positive2 = nn.functional.cosine_similarity(anchor_output, positive_output2, dim=1)


# Cosine similarity between anchor and negatives
cosine_sim_negative1 = nn.functional.cosine_similarity(anchor_output, negative_output1, dim=1)
cosine_sim_negative2 = nn.functional.cosine_similarity(anchor_output, negative_output2, dim=1)

# Compute Triplet Loss (for positive1-negative1 and positive2-negative2)
triplet_loss1 = torch.clamp(margin - cosine_sim_positive1 + cosine_sim_negative1, min=0)
triplet_loss2 = torch.clamp(margin - cosine_sim_positive2 + cosine_sim_negative2, min=0)

# Average the losses
triplet_loss = (triplet_loss1 + triplet_loss2).mean()

# Backpropagation
triplet_loss.backward()

# Print debugging information
print("Anchor output:", anchor_output)
print("Positive 1 output:", positive_output1)
print("Positive 2 output:", positive_output2)
print("Negative 1 output:", negative_output1)
print("Negative 2 output:", negative_output2)
print("Cosine similarity positive 1:", cosine_sim_positive1)
print("Cosine similarity positive 2:", cosine_sim_positive2)
print("Cosine similarity negative 1:", cosine_sim_negative1)
print("Cosine similarity negative 2:", cosine_sim_negative2)
print("Triplet loss:", triplet_loss)


Anchor output: tensor([[-0.1423, -0.1539,  0.1909]], grad_fn=<AddmmBackward0>)
Positive 1 output: tensor([[-0.2249,  0.2396,  0.2570]], grad_fn=<AddmmBackward0>)
Positive 2 output: tensor([[-0.6124,  0.3555,  0.0086]], grad_fn=<AddmmBackward0>)
Negative 1 output: tensor([[-0.4607,  0.0852,  0.0928]], grad_fn=<AddmmBackward0>)
Negative 2 output: tensor([[-0.3622,  0.3388,  0.0395]], grad_fn=<AddmmBackward0>)
Cosine similarity positive 1: tensor([0.3736], grad_fn=<SumBackward1>)
Cosine similarity positive 2: tensor([0.1697], grad_fn=<SumBackward1>)
Cosine similarity negative 1: tensor([0.5182], grad_fn=<SumBackward1>)
Cosine similarity negative 2: tensor([0.0493], grad_fn=<SumBackward1>)
Triplet loss: tensor(2.0242, grad_fn=<MeanBackward0>)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  # Import tqdm for progress bar

# Define the margin for triplet loss
margin = 1.0
num_epochs = 1000  # Number of iterations/epochs to run
learning_rate = 0.001  # Learning rate for optimizer
batch_size = 16  # Batch size for training

# Initialize models (Make sure TowerOne and TowerTwo are defined)
tower_one = TowerOne()
tower_two = TowerTwo()

# Define optimizer
optimizer = optim.Adam(list(tower_one.parameters()) + list(tower_two.parameters()), lr=learning_rate)

# TripletMarginWithDistanceLoss with cosine similarity as the distance function
triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1.0 - nn.functional.cosine_similarity(x, y),
    margin=margin
)

# Training loop with tqdm for progress tracking
with tqdm(total=num_epochs, desc="Training Progress", unit="epoch") as pbar:
    for epoch in range(num_epochs):
        # Generate random batches of video inputs (batch size anchor, 2 positive, 2 negative)
        anchor_query = torch.rand(batch_size, 64)    # Anchor batch
        positive_video1 = torch.rand(batch_size, 64) # Positive batch 1
        positive_video2 = torch.rand(batch_size, 64) # Positive batch 2
        negative_video1 = torch.rand(batch_size, 64) # Negative batch 1
        negative_video2 = torch.rand(batch_size, 64) # Negative batch 2
        
        # Zero gradients (reset gradients before backpropagation)
        optimizer.zero_grad()

        # Forward pass through the tower for each video batch
        anchor_output = tower_one(anchor_query)          # Anchor
        positive_output1 = tower_two(positive_video1)    # Positive sample 1
        positive_output2 = tower_two(positive_video2)    # Positive sample 2
        negative_output1 = tower_two(negative_video1)    # Negative sample 1
        negative_output2 = tower_two(negative_video2)    # Negative sample 2

        # Calculate triplet loss using custom cosine similarity distance
        triplet_loss1 = triplet_loss_fn(anchor_output, positive_output1, negative_output1)
        triplet_loss2 = triplet_loss_fn(anchor_output, positive_output2, negative_output2)

        # Average the triplet losses
        triplet_loss = (triplet_loss1 + triplet_loss2) / 2

        # Backpropagation
        triplet_loss.backward()

        # Update the model parameters
        optimizer.step()

        # Update tqdm description with current loss
        pbar.set_postfix({"Triplet Loss": triplet_loss.item()})
        pbar.update(1)  # Increment the progress bar

print("Training complete.")


Training Progress: 100%|██████████| 1000/1000 [00:02<00:00, 423.17epoch/s, Triplet Loss=1.04]


Training complete.
